In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("MyApp") \
    .master("local[*]") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/06/09 12:10:58 WARN Utils: Your hostname, kirans-mac.local, resolves to a loopback address: 127.0.0.1; using 172.18.197.149 instead (on interface en0)
25/06/09 12:10:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/09 12:10:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Create RDD

In [3]:
ord = spark.sparkContext.textFile('/Users/kiranchinta/Downloads/RetailDB SalesData/Orders')
ordItems = spark.sparkContext.textFile('/Users/kiranchinta/Downloads/RetailDB SalesData/Order_items')

In [4]:
# order items
# itemid, item_orider_id, item_product_id,item_quantity, item_revenue, item_price

In [20]:
ord.take(5)

['1,2013-07-25 00:00:00.0,11599,CLOSED',
 '2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT',
 '3,2013-07-25 00:00:00.0,12111,COMPLETE',
 '4,2013-07-25 00:00:00.0,8827,CLOSED',
 '5,2013-07-25 00:00:00.0,11318,COMPLETE']

### sort by customer id

In [21]:
ordpair =  ord.map(lambda x : (x.split(',')[2],x))
ordpair.take(5)

[('11599', '1,2013-07-25 00:00:00.0,11599,CLOSED'),
 ('256', '2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT'),
 ('12111', '3,2013-07-25 00:00:00.0,12111,COMPLETE'),
 ('8827', '4,2013-07-25 00:00:00.0,8827,CLOSED'),
 ('11318', '5,2013-07-25 00:00:00.0,11318,COMPLETE')]

In [22]:
ordpair.sortByKey().take(10)

[('1', '22945,2013-12-13 00:00:00.0,1,COMPLETE'),
 ('10', '45239,2014-05-01 00:00:00.0,10,COMPLETE'),
 ('10', '56133,2014-07-15 00:00:00.0,10,COMPLETE'),
 ('100', '6641,2013-09-05 00:00:00.0,100,COMPLETE'),
 ('100', '15045,2013-10-28 00:00:00.0,100,PROCESSING'),
 ('100', '22395,2013-12-09 00:00:00.0,100,CANCELED'),
 ('100', '28477,2014-01-16 00:00:00.0,100,COMPLETE'),
 ('100', '54995,2014-07-08 00:00:00.0,100,COMPLETE'),
 ('100', '62907,2014-02-06 00:00:00.0,100,PENDING_PAYMENT'),
 ('100', '64426,2014-04-06 00:00:00.0,100,PENDING')]

In [23]:
ordpair.sortByKey(ascending=False).take(10)

[('9999', '3471,2013-08-14 00:00:00.0,9999,COMPLETE'),
 ('9999', '41855,2014-04-09 00:00:00.0,9999,SUSPECTED_FRAUD'),
 ('9999', '52395,2014-06-18 00:00:00.0,9999,CLOSED'),
 ('9999', '55071,2014-07-08 00:00:00.0,9999,COMPLETE'),
 ('9999', '62988,2014-02-09 00:00:00.0,9999,PENDING'),
 ('9998', '11553,2013-10-04 00:00:00.0,9998,PENDING_PAYMENT'),
 ('9998', '32253,2014-02-09 00:00:00.0,9998,PENDING_PAYMENT'),
 ('9998', '41298,2014-04-04 00:00:00.0,9998,PROCESSING'),
 ('9998', '59429,2013-09-29 00:00:00.0,9998,ON_HOLD'),
 ('9998', '59713,2013-10-10 00:00:00.0,9998,CANCELED')]

### sort by item customer, status

In [24]:
ordpair =  ord.map(lambda x : ((x.split(',')[2],x.split(',')[3]),x))
ordpair.take(5)

[(('11599', 'CLOSED'), '1,2013-07-25 00:00:00.0,11599,CLOSED'),
 (('256', 'PENDING_PAYMENT'), '2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT'),
 (('12111', 'COMPLETE'), '3,2013-07-25 00:00:00.0,12111,COMPLETE'),
 (('8827', 'CLOSED'), '4,2013-07-25 00:00:00.0,8827,CLOSED'),
 (('11318', 'COMPLETE'), '5,2013-07-25 00:00:00.0,11318,COMPLETE')]

In [25]:
ordpair.sortByKey().take(10)

[(('1', 'COMPLETE'), '22945,2013-12-13 00:00:00.0,1,COMPLETE'),
 (('10', 'COMPLETE'), '45239,2014-05-01 00:00:00.0,10,COMPLETE'),
 (('10', 'COMPLETE'), '56133,2014-07-15 00:00:00.0,10,COMPLETE'),
 (('100', 'CANCELED'), '22395,2013-12-09 00:00:00.0,100,CANCELED'),
 (('100', 'COMPLETE'), '6641,2013-09-05 00:00:00.0,100,COMPLETE'),
 (('100', 'COMPLETE'), '28477,2014-01-16 00:00:00.0,100,COMPLETE'),
 (('100', 'COMPLETE'), '54995,2014-07-08 00:00:00.0,100,COMPLETE'),
 (('100', 'PENDING'), '64426,2014-04-06 00:00:00.0,100,PENDING'),
 (('100', 'PENDING_PAYMENT'),
  '62907,2014-02-06 00:00:00.0,100,PENDING_PAYMENT'),
 (('100', 'PROCESSING'), '15045,2013-10-28 00:00:00.0,100,PROCESSING')]

### takeordered()

In [40]:
prod = spark.sparkContext.textFile('/Users/kiranchinta/Downloads/RetailDB SalesData/Products')

In [41]:
prod.take(5)

['1,2,Quest Q64 10 FT. x 10 FT. Slant Leg Instant U,,59.98,http://images.acmesports.sports/Quest+Q64+10+FT.+x+10+FT.+Slant+Leg+Instant+Up+Canopy',
 "2,2,Under Armour Men's Highlight MC Football Clea,,129.99,http://images.acmesports.sports/Under+Armour+Men%27s+Highlight+MC+Football+Cleat",
 "3,2,Under Armour Men's Renegade D Mid Football Cl,,89.99,http://images.acmesports.sports/Under+Armour+Men%27s+Renegade+D+Mid+Football+Cleat",
 "4,2,Under Armour Men's Renegade D Mid Football Cl,,89.99,http://images.acmesports.sports/Under+Armour+Men%27s+Renegade+D+Mid+Football+Cleat",
 '5,2,Riddell Youth Revolution Speed Custom Footbal,,199.99,http://images.acmesports.sports/Riddell+Youth+Revolution+Speed+Custom+Football+Helmet']

In [42]:
# product_id ,category_id, , product_name, product_description, price, image

### sort by product price

In [47]:
prod = prod.filter(lambda x: x.split(',')[4]!= '')
prodpair = prod.map(lambda x : (float(x.split(',')[4]),x))
prodpair.take(5)

[(59.98,
  '1,2,Quest Q64 10 FT. x 10 FT. Slant Leg Instant U,,59.98,http://images.acmesports.sports/Quest+Q64+10+FT.+x+10+FT.+Slant+Leg+Instant+Up+Canopy'),
 (129.99,
  "2,2,Under Armour Men's Highlight MC Football Clea,,129.99,http://images.acmesports.sports/Under+Armour+Men%27s+Highlight+MC+Football+Cleat"),
 (89.99,
  "3,2,Under Armour Men's Renegade D Mid Football Cl,,89.99,http://images.acmesports.sports/Under+Armour+Men%27s+Renegade+D+Mid+Football+Cleat"),
 (89.99,
  "4,2,Under Armour Men's Renegade D Mid Football Cl,,89.99,http://images.acmesports.sports/Under+Armour+Men%27s+Renegade+D+Mid+Football+Cleat"),
 (199.99,
  '5,2,Riddell Youth Revolution Speed Custom Footbal,,199.99,http://images.acmesports.sports/Riddell+Youth+Revolution+Speed+Custom+Football+Helmet')]

In [48]:
prodpair.sortByKey().take(5)

[(0.0,
  "38,3,Nike Men's Hypervenom Phantom Premium FG Socc,,0.0,http://images.acmesports.sports/Nike+Men%27s+Hypervenom+Phantom+Premium+FG+Soccer+Cleat"),
 (0.0,
  "388,18,Nike Men's Hypervenom Phantom Premium FG Socc,,0.0,http://images.acmesports.sports/Nike+Men%27s+Hypervenom+Phantom+Premium+FG+Soccer+Cleat"),
 (0.0,
  "414,19,Nike Men's Hypervenom Phantom Premium FG Socc,,0.0,http://images.acmesports.sports/Nike+Men%27s+Hypervenom+Phantom+Premium+FG+Soccer+Cleat"),
 (0.0,
  "517,24,Nike Men's Hypervenom Phantom Premium FG Socc,,0.0,http://images.acmesports.sports/Nike+Men%27s+Hypervenom+Phantom+Premium+FG+Soccer+Cleat"),
 (0.0,
  "547,25,Nike Men's Hypervenom Phantom Premium FG Socc,,0.0,http://images.acmesports.sports/Nike+Men%27s+Hypervenom+Phantom+Premium+FG+Soccer+Cleat")]

In [49]:
prodpair.sortByKey(False).take(5)

[(1999.99,
  '208,10,SOLE E35 Elliptical,,1999.99,http://images.acmesports.sports/SOLE+E35+Elliptical'),
 (1799.99,
  '66,4,SOLE F85 Treadmill,,1799.99,http://images.acmesports.sports/SOLE+F85+Treadmill'),
 (1799.99,
  '199,10,SOLE F85 Treadmill,,1799.99,http://images.acmesports.sports/SOLE+F85+Treadmill'),
 (1799.99,
  '496,22,SOLE F85 Treadmill,,1799.99,http://images.acmesports.sports/SOLE+F85+Treadmill'),
 (1099.99,
  '1048,47,"Spalding Beast 60"" Glass Portable Basketball ",,1099.99,http://images.acmesports.sports/Spalding+Beast+60%22+Glass+Portable+Basketball+Hoop')]

In [50]:
prod.takeOrdered(5,lambda x : float(x.split(',')[4]))

["38,3,Nike Men's Hypervenom Phantom Premium FG Socc,,0.0,http://images.acmesports.sports/Nike+Men%27s+Hypervenom+Phantom+Premium+FG+Soccer+Cleat",
 "388,18,Nike Men's Hypervenom Phantom Premium FG Socc,,0.0,http://images.acmesports.sports/Nike+Men%27s+Hypervenom+Phantom+Premium+FG+Soccer+Cleat",
 "414,19,Nike Men's Hypervenom Phantom Premium FG Socc,,0.0,http://images.acmesports.sports/Nike+Men%27s+Hypervenom+Phantom+Premium+FG+Soccer+Cleat",
 "517,24,Nike Men's Hypervenom Phantom Premium FG Socc,,0.0,http://images.acmesports.sports/Nike+Men%27s+Hypervenom+Phantom+Premium+FG+Soccer+Cleat",
 "547,25,Nike Men's Hypervenom Phantom Premium FG Socc,,0.0,http://images.acmesports.sports/Nike+Men%27s+Hypervenom+Phantom+Premium+FG+Soccer+Cleat"]

In [51]:
prod.takeOrdered(5,lambda x : -float(x.split(',')[4]))

['208,10,SOLE E35 Elliptical,,1999.99,http://images.acmesports.sports/SOLE+E35+Elliptical',
 '66,4,SOLE F85 Treadmill,,1799.99,http://images.acmesports.sports/SOLE+F85+Treadmill',
 '199,10,SOLE F85 Treadmill,,1799.99,http://images.acmesports.sports/SOLE+F85+Treadmill',
 '496,22,SOLE F85 Treadmill,,1799.99,http://images.acmesports.sports/SOLE+F85+Treadmill',
 '1048,47,"Spalding Beast 60"" Glass Portable Basketball ",,1099.99,http://images.acmesports.sports/Spalding+Beast+60%22+Glass+Portable+Basketball+Hoop']

# grouped ranking

In [52]:
prodF = prod.filter(lambda x : int(x.split(',')[1]) in [2,3,4] and (int(x.split(',')[0]) in [1,2,3,4,5,25,26,27,28,29,49,50,51,52,53]))

In [53]:
prodF.take(5)

['1,2,Quest Q64 10 FT. x 10 FT. Slant Leg Instant U,,59.98,http://images.acmesports.sports/Quest+Q64+10+FT.+x+10+FT.+Slant+Leg+Instant+Up+Canopy',
 "2,2,Under Armour Men's Highlight MC Football Clea,,129.99,http://images.acmesports.sports/Under+Armour+Men%27s+Highlight+MC+Football+Cleat",
 "3,2,Under Armour Men's Renegade D Mid Football Cl,,89.99,http://images.acmesports.sports/Under+Armour+Men%27s+Renegade+D+Mid+Football+Cleat",
 "4,2,Under Armour Men's Renegade D Mid Football Cl,,89.99,http://images.acmesports.sports/Under+Armour+Men%27s+Renegade+D+Mid+Football+Cleat",
 '5,2,Riddell Youth Revolution Speed Custom Footbal,,199.99,http://images.acmesports.sports/Riddell+Youth+Revolution+Speed+Custom+Football+Helmet']

In [55]:
prodFGroupBy = prodF.map(lambda x : ( int(x.split(',')[1]), x)).groupByKey()
prodFGroupBy.take(5)

[(2, <pyspark.resultiterable.ResultIterable at 0x104803ed0>),
 (4, <pyspark.resultiterable.ResultIterable at 0x1048c88d0>),
 (3, <pyspark.resultiterable.ResultIterable at 0x1059f3610>)]

In [59]:
first = prodFGroupBy.first()
first

(2, <pyspark.resultiterable.ResultIterable at 0x1048c8310>)

In [60]:
sorted(first[1],key = lambda x : float(x.split(',')[4]),reverse=True)

['5,2,Riddell Youth Revolution Speed Custom Footbal,,199.99,http://images.acmesports.sports/Riddell+Youth+Revolution+Speed+Custom+Football+Helmet',
 "2,2,Under Armour Men's Highlight MC Football Clea,,129.99,http://images.acmesports.sports/Under+Armour+Men%27s+Highlight+MC+Football+Cleat",
 "3,2,Under Armour Men's Renegade D Mid Football Cl,,89.99,http://images.acmesports.sports/Under+Armour+Men%27s+Renegade+D+Mid+Football+Cleat",
 "4,2,Under Armour Men's Renegade D Mid Football Cl,,89.99,http://images.acmesports.sports/Under+Armour+Men%27s+Renegade+D+Mid+Football+Cleat",
 '1,2,Quest Q64 10 FT. x 10 FT. Slant Leg Instant U,,59.98,http://images.acmesports.sports/Quest+Q64+10+FT.+x+10+FT.+Slant+Leg+Instant+Up+Canopy']